In [ ]:
## Generate the summary metrics using the generated dataset

In [ ]:
! pip install --user pandas pyarrow fastparquet

In [ ]:
! pip install --user imblearn

In [ ]:
! pip install pyyaml

In [2]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/repos/smote_msfb/helper_functions')
from summarize_dataset_with_hubness import summarize_dataset_with_hubness
from create_folder_if_not_exists import create_folder_if_not_exists
from run_single_experiment import run_single_experiment

In [3]:
path = "/repos/smote_msfb/public_datasets/enron/"

In [4]:
## Create summary metric dataset for this public dataset
df = pd.read_parquet( path + "processed_dataset.parquet" , engine="pyarrow")

In [6]:
# Suppose df is your dataset already loaded into memory
# Features: f_*
feature_cols = [col for col in df.columns if col.startswith("f_")]

# Targets: target_*
target_cols = [col for col in df.columns if col.startswith("target_")]

In [ ]:
for target in target_cols: #[13:14]:
    X = df[feature_cols]
    y = df[[target]]  # keep as DataFrame
    combined_df = pd.concat([X, y], axis=1).reset_index(drop=True)
    
    # Rename target column to "target"
    combined_df = combined_df.rename(columns={target: "target"})    
    combined_df = combined_df.astype(int)
       
    print(combined_df['target'].value_counts())
    
    output_loc = "/domino/datasets/local/smote_msfb/public_datasets/enron/" + "/" + str(target) + "/"
    create_folder_if_not_exists(output_loc)
    
    # Redirect stdout to file
    log_file_path = output_loc + "output.txt"
    with open(log_file_path, "w") as f:
        # Temporarily redirect stdout
        old_stdout = sys.stdout
        sys.stdout = f
        try:
            run_single_experiment(output_loc, combined_df, mode="new")
        finally:
            # Restore stdout even if experiment fails
            sys.stdout = old_stdout
        